## Monolingual Norec experiments with new BERT models
Targeted sentiment analysis with simpletransformers for sequence tagging. Highly recommend to do this in a dedicated Python environment. You need PyTorch to interact with Cuda, and Simpletransformers to interact with pytorch, and you need the right Python version to support this chain. I suggest you begin with having a Cuda version that is listed in the pytorch installation guide, and take it from there.  

### CPU only.
For exploring norBERT with SimpleTransformers. There is, as per January 2022, something odd when using norBERT with Simpletransformers. This script is to find out more. Check the loss tat is returned for norBERT

In [1]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import pandas as pd
import pickle as pk
import os
import random
import json
import time, datetime
random.seed(64)
import re
from helpers import *

print(f"Cuda: {torch.cuda.is_available()}")

Cuda: False


## Prepare the data.
I collect the experiments to run as a list of tuples. Each experiment consists of training set and testing set, and a list of tags. This list of tags is used only for checking that the files have the tags we expect. Highly recommend to do this checking, that we have the expected tags in the datasets. Token and tags need to be spaceseparated. 
Training on 400 sentences, since we are on CPU

In [2]:
# Origianlly made for multiple datasets
datasets = [('data/spaceseparated/norec_fine_scaled/scaled_norec_400.conll', 'data/spaceseparated/norec_fine/norec_fine_dev.conll', ['I-targ-negative', 'I-targ-positive',
 'B-targ-negative', 'B-targ-positive', 'O'])]
train_path, dev_path, tags = datasets[0]
# Check that the tags are consistent 
# You may delete the following, and the notebook will then run without the helpers.py
for path in [dev_path,train_path]:
    with open (path) as rf:
        text = rf.read()
        assert tagsset(text, separator = " ") == set(tags), tagsset(text, separator = " ")
print("Tags checked OK")

Tags checked OK


## Perform the fine-tuning
Note that this script does not automatically  create the folders needed to save the model and to record the output. I recommend you run the following cell with 1 epoch to see that this works, befor setting it back to 8 or whatever you consider to be adequate. 3 should be enough, but I got a litte better result with 8 so I kept that.

Simpletransformers has included the code for wandb but I have not tried to connect and use that what is supposed to be a great reporting and logging tool.

Note that if you run many epochs and save the models, you will need a lot of space.

In [3]:
# Run bert multilingual with the data from previous cell

family = "bert"
transformersmodels = ['bert-base-multilingual-cased' , 'NbAiLab/nb-bert-base',  'ltgoslo/norbert' ]
results = []

for transformersmodel in transformersmodels:
    model_args = NERArgs() 
    model_args.train_batch_size = 12
    model_args.num_train_epochs = 2
    model_args.weight_decay = 0.001
    model_args.overwrite_output_dir = True
    model_args.silent = False
    model_args.save_model_every_epoch = False
    model_args.save_eval_checkpoints = True
    model_args.save_steps = -1
    model_args.warmup_ratio = 0

    model = NERModel(family,transformersmodel , labels = tags,args=model_args, use_cuda=False)

    out_d = "outputs/simpletransformers/"+transformersmodel+"_"+train_path.split("/")[-2]

    model.train_model(train_path, output_dir= out_d)
    print(transformersmodel, "Done training")

    result, model_outputs, predictions = model.eval_model(dev_path)
    result["transformer_model"] = transformersmodel
    for k, v in result.items():
        print(transformersmodel, k, v)
    predictions, raw_outputs = model.predict (["Mannen på scenen synger stygt", "Damen på scenen synger stygt" , "Disse bilene har et fantastisk veigrep"])
    for sentence in predictions:
        print(sentence)
    results.append(result)


df = pd.DataFrame.from_dict(results)



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

bert-base-multilingual-cased Done training


Running Evaluation: 100%|██████████| 192/192 [01:53<00:00,  1.69it/s]


bert-base-multilingual-cased eval_loss 0.24955419702261375
bert-base-multilingual-cased precision 0.2727272727272727
bert-base-multilingual-cased recall 0.05131128848346636
bert-base-multilingual-cased f1_score 0.08637236084452976
bert-base-multilingual-cased transformer_model bert-base-multilingual-cased


Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  3.89it/s]


...

ltgoslo/norbert Done training


Running Evaluation: 100%|██████████| 192/192 [01:52<00:00,  1.70it/s]


ltgoslo/norbert eval_loss 0.3231751738348976
ltgoslo/norbert precision 0.0
ltgoslo/norbert recall 0.0
ltgoslo/norbert f1_score 0.0
ltgoslo/norbert transformer_model ltgoslo/norbert


/home/egil/miniconda3/envs/st_cpu/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  4.78it/s]

[{'Mannen': 'O'}, {'på': 'O'}, {'scenen': 'O'}, {'synger': 'O'}, {'stygt': 'O'}]
[{'Damen': 'O'}, {'på': 'O'}, {'scenen': 'O'}, {'synger': 'O'}, {'stygt': 'O'}]
[{'Disse': 'O'}, {'bilene': 'O'}, {'har': 'O'}, {'et': 'O'}, {'fantastisk': 'O'}, {'veigrep': 'O'}]


In [4]:
# Run all above
df

,eval_loss,precision,recall,f1_score,transformer_model
0,0.249554,0.272727,0.051311,0.086372,bert-base-multilingual-cased
1,0.252837,0.174129,0.039909,0.064935,NbAiLab/nb-bert-base
2,0.323175,0.000000,0.000000,0.000000,ltgoslo/norbert


In [5]:
df.to_csv(f"summaries/norec_3berts_cpu.csv")